In [20]:
import os
import re
import sys
import time
import pickle
import pprint
import requests
import datetime
import numpy as np
import pandas as pd
from selenium import webdriver
from collections import Counter
from bs4 import BeautifulSoup as bs
from get_credentials import get_creds
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (NoSuchElementException,
                                        ElementNotInteractableException,
                                        ElementClickInterceptedException) 

In [21]:
if os.path.exists('credentials.pickle'):
    with open('credentials.pickle','rb') as f:  
        credentials = pickle.load(f)
else:
    get_creds()
    with open('credentials.pickle','rb') as f:  
        credentials = pickle.load(f)

In [22]:
TEAMS_PATH =  (r'https://login.microsoftonline.com/common/oauth2/authorize?'
               'response_type=id_token&client_id=5e3ce6c0-2b1f-4285-8d4b-75'
               'ee78787346&redirect_uri=https%3A%2F%2Fteams.microsoft.com%2F'
               'go&state=00b8dd1b-8f5d-4eb3-8f77-5c76ff40e3b2&&client-request'
               '-id=d2afe301-0371-4fb6-b185-1ad0345268b5&x-client-SKU=Js&x-cli'
               'ent-Ver=1.0.9&nonce=3afa9b8e-d754-4aae-bb73-ab6fd6bc2f33&domain_hint=')

EXECUTABLE_PATH = credentials['EXECUTABLE_PATH']
EMAIL_ID = credentials['Email_or_phone_no']
PASSWORD = credentials['Password']
TEAM = "DRISHTI-ML"

In [23]:
status = [
    'INSTALLATION IN PROGRESS',
    'INSTALLATION DONE',
    
    'ASSIGNMENT1 IN PROGRESS',
    'ASSIGNMENT1 DONE',
    
    'ASSIGNMENT2 IN PROGRESS',
    'ASSIGNMENT2 DONE',
    
    'ASSIGNMENT3 IN PROGRESS',
    'ASSIGNMENT3 DONE',
    
    'ASSIGNMENT4 IN PROGRESS',
    'ASSIGNMENT4 DONE',         
]

In [24]:
def get_bot(bot,refresh = True):
    flag = True
    if bot == 1: #main bot
        print('getting main bot')
        BOT = ('https://teams.microsoft.com/_#/school/conversations/'
               'AUTO?threadId=19:72818b8c80fb4045bf22a2f19e8af04b@thr'
               'ead.tacv2&ctx=channel')
    elif bot == -1: #private bot
        print('getting private bot')
        BOT = ('https://teams.microsoft.com/_#/school/conversations/'
               'BOT?threadId=19:84124691e65d4ebabaee9d78e41ee04f@thre'
               'ad.tacv2&ctx=channel')
    else:
        flag = False
        print("INVALID BOT CHOICE!!")
    if flag:
        time.sleep(2)
        driver.get(BOT)
        time.sleep(2)
        if refresh:
            driver.refresh()
    else:
        sys.exit()

In [25]:
def get_message_stuff():
    #get all message_bodies
    message_bodies = driver.find_elements_by_id('messageBody')
    #get page_source and findall message_ids
    message_ids = re.findall(r'<div id="(m[0-9]*)"',str(driver.page_source))
    return message_bodies,message_ids

In [26]:
# #grab the message
# def edit(message_bodies,message_ids,bot):
#     try:
#         print('trying to edit')
#         time.sleep(3)
#         message_bodies,message_ids = get_message_stuff()
#         print('message ids:',message_ids)
#         message_to_edit = message_bodies[INDEX_OF_MESSAGE]
#         print(f'message to edit: {message_to_edit}\n Scrolling message into view....')
#         driver.execute_script("return arguments[0].scrollIntoView(true);", message_to_edit)
#         print('scrolled successfully, trying to click on body...')
#         driver.find_element_by_xpath("//body").click() #click somewhere else on the body
#         time.sleep(0.5)
#         print('hovering over message..')
#         hover = ActionChains(driver).move_to_element(message_to_edit)
#         hover.perform()
#         time.sleep(0.5)
#         print('hovered over message..')
#         driver.find_element_by_css_selector('.message-actions-more').click()#click on "..."
#         time.sleep(0.5)
#         driver.find_element_by_css_selector("li.message-action-overflow-menu-item:nth-child(2) > button:nth-child(1) > span:nth-child(2)").click()#select edit        
#     except:
#         get_bot(bot = bot,refresh = True)
#         message_bodies,message_ids = get_message_stuff()
#         message_bodies,message_ids = edit(message_bodies,message_ids,bot)
#     return message_bodies,message_ids

In [27]:
def generate_string(students):
    student_info = ''
    student_list = [[x]+students[x] for x in students]
    student_list = sorted(student_list,key=lambda student_list: int(student_list[2]), reverse=True)
    for i,s in reversed(list(enumerate(status))):
        sub_list = [x[:-1] for x in student_list if x[-1]==f'{str(i)}']
        if not sub_list==[]:
            student_info+="\n\n>>> "+s+"\n\n"
        student_info += '\n'.join([' '.join(ele) for ele in sub_list]) 
    return student_info

In [28]:
def new_student_entry():
    print('trying to get new message metadata')
    message_bodies,message_ids = get_message_stuff()
    
    name,uid,cur_status = None,None,None
    inner_html = driver.find_element_by_id(message_ids[-1]).get_attribute('innerHTML')
    soup = bs(inner_html,'html.parser')
    search_string = soup.findAll("div", {"class": "screen-reader-text"})
    cur_status = [a.text for a in search_string][0].replace('\n','').split(',')[0]

    try:
        for k in range(-1,-900,-1): # climb up till you find name and uid
            try:
                inner_html = driver.find_element_by_id(message_ids[k]).get_attribute('innerHTML')    
                (name,uid)= re.findall('alt="Profile picture of ([A-Z ]*)." upn="([0-9A-Z]*)@svnitsuratg.onmicrosoft.com"',inner_html)[0]
                name = name.split()[-1]
                break
            except IndexError:
                continue
    except :
        print('failed')
        time.sleep(0.1)
        new_student_entry()
    print('success',name,uid,cur_status)
   
    assert  name != None
    assert  uid != None
    assert  cur_status != None
    return name,uid,cur_status

In [29]:
def refresh_status(bot):
    print(f'refreshing status for bot{bot}')
    students = update_in_code()
    print('clicking on edit')
    message_bodies,message_ids = get_message_stuff()
    edit(message_bodies,message_ids,bot=bot)
    print('dumping payload')
    dump_payload(students,bot) 

In [30]:
def remove(students,uid,bot):
    if uid in students:
        del students[uid]
        update_local()
        students = update_in_code()
        refresh_status(bot)
    return students

In [31]:
def update_local():
    with open('students.pickle','wb') as f:
        pickle.dump(students,f)

def update_in_code():
    with open('students.pickle','rb') as f:
        students = pickle.load(f)
    return students

In [32]:
def send_reply(payload,bot):
    try:
        #click on reply button
        try:
            driver.find_elements_by_class_name('thread-action-reply-message')[-1].click()
        except ElementClickInterceptedException:
            time.sleep(2)
            send_reply(payload,bot)
        in_htlm = str(driver.find_element_by_class_name('ts-edit-box').get_attribute("innerHTML"))
        cke_contents_id_name = re.findall(r'<div id="(cke_[0-9]*_contents)"',in_htlm)[0]
        cke_wysiwyg_div  = f"#{cke_contents_id_name} > div:nth-child(1)"
        cke_element = driver.find_element_by_css_selector(cke_wysiwyg_div)

        #load data into clipboard
        df = pd.DataFrame([payload])
        df.to_clipboard(index=False,header=False)

        #clear prev text
        cke_element.send_keys(Keys.CONTROL, "a")
        cke_element.send_keys(Keys.DELETE)

        time.sleep(0.5)

        #dump payload
        cke_element.send_keys(Keys.CONTROL, "v")
        cke_element.send_keys(Keys.RETURN)
    #     print('success!')
    except:
        get_bot(bot = bot,refresh = True)
        time.sleep(4)
        send_reply(payload,bot)

In [33]:
def reply(command,bot,*args):
    
    def add_prefix(string):
        string = "AUTO =>\n"+string
        return string
    
    if command == "STATS":
        departments = [re.split('U19([A-Z][A-Z])',key)[1] for key in students]
        percentages = list(map(lambda x: (x/len(students))*100,list(Counter(departments).values())))
        percentages = [str(x) for x in list(np.round(percentages,3))]
        departments_uniq = list(Counter(departments).keys())
        stats = '\n'.join([departments_uniq[i]+' --> '+percentages[i]+'%' for i in range(0,len(departments_uniq)) ])

        s = [students[key][1] for key in students]
        keys = list(Counter(s).keys())
        values = list(Counter(s).values())
        percentages = list(map(lambda x: (x/len(students))*100,list((Counter(s).values()))))
        percentages = [str(x) for x in list(np.round(percentages,3))]
        stats2 = '\n'.join([status[int(key)]+' --> '+str(values[i])+' --> '+ percentages[i] +'%' for i,key in enumerate(keys)])

        stats = f"\nActive Students: {len(students)}\n\n----Assignment wise----\n\n"+stats2+"\n\n----Department wise----\n\n"+stats
        payload = add_prefix(stats)
        send_reply(payload,bot)
        
    elif command == "STOP":
        send_reply(add_prefix("SHUTTING DOWN. GOODBYE :("),bot)
        
    elif command == "SET_CHECK_PERIOD":
        send_reply(add_prefix("CHECK_PERIOD =="+args[0]),bot)
        
    elif command == "REMOVE":
        send_reply(add_prefix("REMOVED "+args[0]),bot)
        
    elif command == "REFRESH":
        send_reply(add_prefix("REFRESHED"),bot)
    
    elif command == "STUDENT":
        send_reply(add_prefix(f"Hi {args[0]} you have been added to {status[int(args[1])]}"),bot)
        
    elif command == "SWITCH_BOT":
        if bot == 1:
            send_reply(add_prefix("SWITCHING TO PRIVATE BOT"),bot)
        else:
            send_reply(add_prefix("SWITCHING TO MAIN BOT"),bot)
            
    elif command == "HI":
        send_reply(add_prefix(f"Started at: {start}. Running for {str(datetime.timedelta(seconds=int(time.time()-start_mark)))}"),bot)
        
    else: #incorrect command
        send_reply(add_prefix("INVALID COMMAND"),bot)

In [34]:
def valid(students,uid,name,cur_status,bot):
    print('status valid')

    #assign students[uid] to fresh values
    #and save students locally
    students[uid]=[name,cur_status]
    message_bodies,message_ids = get_message_stuff()
    update_local()     
    update_in_code()
    print('clicking on edit')
    message_bodies,message_ids = edit(message_bodies,message_ids,bot=bot)
    print('dumping payload')
    dump_payload(students,bot)     
    
    reply('STUDENT',bot,name,cur_status)

In [332]:
def detect_change(num_of_messages,students,bot,sleep_time=1):
    while True:
        message_bodies,message_ids = get_message_stuff() 
        command=None
        print(num_of_messages,len(message_ids))
        
        if num_of_messages > len(message_ids):
            num_of_messages = len(message_ids)
            
        if len(message_ids) > num_of_messages + 1:
            num_of_messages = len(message_ids)
            
        
        #check if num_of_messages is greater that what it was
        if num_of_messages < len(message_ids):
            num_of_messages = len(message_ids)       
            print('change detected')
            time.sleep(0.2)
            
            #get student data
            name,uid,cur_status = new_student_entry()
            
#             if cur_status[-1] == 'HI':
#                 reply(cur_status,bot)
#                 num_of_messages+=1
            
            
            #check if ME, and if ADMIN_COMMAND
            if name=="ADITYA" and uid=="U18EC013" and cur_status.split(' ')[0]=="AUTO":
                cur_status_split = cur_status.split(' ')
                if len(cur_status_split)== 2:
                    command = cur_status_split[-1]
                elif len(cur_status_split)== 3:
                    command = cur_status_split[-2]
                    
                if command=="STATS":
                    reply(command,bot)
                    num_of_messages+=1
                    
                elif command=="STOP":
                    reply(command,bot)
                    num_of_messages+=1
                    print('STOP command triggered. exiting...')
                    break
                    
                elif command=="SET_CHECK_PERIOD":
                    sleep_time = int(cur_status_split[-1])
                    reply(command,bot,str(sleep_time))
                    num_of_messages+=1
                    
                    
                elif command=="REMOVE":
                    uid = cur_status_split[-1]
                    students = remove(students,uid,bot)
                    reply(command,bot,uid)
                    num_of_messages+=1
                    
                    
                elif command=="REFRESH":
                    refresh_status(bot)
                    reply(command,bot)
                    num_of_messages+=1
                    
                    
                elif command=="SWITCH_BOT":
                    reply(command,bot)
                    num_of_messages+=1
                    break
                    
                else:
                    reply(command,bot)  
                    num_of_messages+=1
                    
            #check if other admins        
            elif ((name=="MEET" and uid=="U18CO075")or(name=="KESHRI" and uid=="U18EC097")or(name=="PARTH" and uid=="U18CO014")) and cur_status.split(' ')[0]=="AUTO":
                cur_status_split = cur_status.split(' ')
                if len(cur_status_split)== 2:
                    command = cur_status_split[-1]
                elif len(cur_status_split)== 3:
                    command = cur_status_split[-2]
                    
                if command=="STATS":
                    reply(command,bot)
                    num_of_messages+=1
                    
                elif command=="SET_CHECK_PERIOD":
                    sleep_time = int(cur_status_split[-1])
                    reply(command,bot,str(sleep_time))
                    num_of_messages+=1
                    
                    
                elif command=="REMOVE":
                    uid = cur_status_split[-1]
                    students = remove(students,uid,bot)
                    reply(command,bot,uid)
                    num_of_messages+=1
                    
                    
                elif command=="REFRESH":
                    refresh_status(bot)
                    reply(command,bot)
                    num_of_messages+=1
                    
                else:
                    reply(command,bot)  
                    num_of_messages+=1

            #checks if student isn't already available and status is a number
            elif (not uid in students) and (cur_status in str(list(range(0,len(status))))):
                valid(students,uid,name,cur_status,bot)
                        
            #check if student status is fresh and is a number
            elif (students[uid]!= [name,cur_status]) and (cur_status in str(list(range(0,len(status))))):
                valid(students,uid,name,cur_status,bot)
        else:
            print('waiting for message...')
            time.sleep(sleep_time)
            
    if command=="SWITCH_BOT":
        bot = bot*-1
        get_bot(bot = bot,refresh = False)
        detect_change(num_of_messages,students,bot=bot)

In [338]:
def dump_payload(students,bot):
    try:
        #get payload
        payload = generate_string(students)
        print('Payload:\n',payload)

        #find cke_element to dump payload into
        message_ids = re.findall(r'<div id="(m[0-9]*)"',str(driver.page_source))
        message_id_outer_htlm = str(driver.find_element_by_id(message_ids[INDEX_OF_MESSAGE]).get_attribute('outerHTML'))
        cke_contents_id_name = re.findall(r'<div id="(cke_[0-9]*_contents)"',message_id_outer_htlm)[0]
        cke_wysiwyg_div  = f"#{cke_contents_id_name} > div:nth-child(1)"
        cke_element = driver.find_element_by_css_selector(cke_wysiwyg_div)

        #load data into clipboard
        df = pd.DataFrame([payload])
        df.to_clipboard(index=False,header=False)

        #clear prev text
        cke_element.send_keys(Keys.CONTROL, "a")
        cke_element.send_keys(Keys.DELETE)

        time.sleep(0.5)

        #dump payload
        cke_element.send_keys(Keys.CONTROL, "v")
        cke_element.send_keys(Keys.RETURN)
        print('success!')
        
    except:
        traceback.print_exc()
        print('something went wrong in dump_payload... refreshing')
        get_bot(bot = bot,refresh = True)
        edit(message_bodies, message_ids, bot)
        time.sleep(5)
        dump_payload(students,bot)

In [339]:
def main(bot,NEW_SESSION=False):
    if NEW_SESSION:
        
        #get teams' login page
        driver.get(TEAMS_PATH)
        time.sleep(1)

        try:
            #try adding email
            email = driver.find_element_by_xpath('//*[@id="i0116"]')
            email.send_keys(EMAIL_ID) 
            email.send_keys(Keys.RETURN)
        except NoSuchElementException:
            #if not found then find and click stored email
            driver.find_element_by_xpath('/html/body/div/form[1]/div/div/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div/div[1]/div/div/div/div[2]/div').click()

        time.sleep(1)

        #enter password
        password = driver.find_element_by_xpath('//*[@id="i0118"]')
        password.send_keys(PASSWORD) 
        time.sleep(0.1)
        password.send_keys(Keys.RETURN)

        time.sleep(2)

        #hit yes
        driver.find_element_by_xpath('//*[@id="idSIButton9"]').click()

        time.sleep(5)

    get_bot(bot=bot,refresh=False)
    
    time.sleep(4)
    
    message_bodies,message_ids = get_message_stuff()
    num_of_messages = len(message_ids)
    print(num_of_messages)
    
    students = update_in_code() 
    detect_change(num_of_messages,students,bot=bot,sleep_time=2)

In [340]:
#grab the message
def edit(message_bodies,message_ids,bot):
#     import pdb;pdb.set_trace()
    try:
#         print('trying to edit')
        time.sleep(3)
        message_bodies,message_ids = get_message_stuff()
#         print('message ids:',message_ids)
        message_to_edit = message_bodies[INDEX_OF_MESSAGE]
#         print(f'message to edit: {message_to_edit}\n Scrolling message into view....')
        driver.execute_script("return arguments[0].scrollIntoView(true);", message_to_edit)
    
        top()
#         print('scrolled successfully, trying to click on body...')
#         driver.find_element_by_xpath("//body").click() #click somewhere else on the body
        time.sleep(0.5)
        message_bodies,message_ids = get_message_stuff()
        message_to_edit = message_bodies[INDEX_OF_MESSAGE]
        print('hovering over message..')
        hover = ActionChains(driver).move_to_element(message_to_edit)
        hover.perform()
        time.sleep(0.5)
        print('hovered over message..')
        driver.find_element_by_css_selector('.message-actions-more').click()#click on "..."
        print('clicked on ...')
        time.sleep(0.5)
        driver.find_element_by_css_selector("li.message-action-overflow-menu-item:nth-child(2) > button:nth-child(1) > span:nth-child(2)").click()#select edit   
        print('clicked on edit.')
    except Exception:
        traceback.print_exc()
        print('failed. trying to get bot...')
        get_bot(bot = bot,refresh = True)
        message_bodies,message_ids = get_message_stuff()
        message_bodies,message_ids = edit(message_bodies,message_ids,bot)
    return message_bodies,message_ids

In [365]:
BOT = -1
INDEX_OF_MESSAGE = 1
NEW_SESSION = False

In [ ]:
#m1591835029919 > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > see-more:nth-child(1) > div:nth-child(1) > div:nth-child(1)

In [366]:
if __name__ == "__main__":
    start = datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')
    start_mark = time.time()
    students = update_in_code()
    if NEW_SESSION:
        driver = webdriver.Firefox(executable_path=EXECUTABLE_PATH)
    main(BOT,NEW_SESSION)

getting private bot
16
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for message...
16 16
waiting for me

KeyboardInterrupt: 

In [353]:
def generate_string(students):
    student_info = ''
    student_list = [[x]+students[x] for x in students]
    student_list = sorted(student_list,key=lambda student_list: int(student_list[2]), reverse=True)
    print(student_list)
    
    for i,s in reversed(list(enumerate(status))):
        sub_list = [x[:-1] for x in student_list if x[-1]==f'{str(i)}']
        print(sub_list)
        if not sub_list==[]:
            student_info+="\n\n>>> "+s+"\n\n"
        student_info += '\n'.join([' '.join(ele) for ele in sub_list]) 
    return student_info

In [363]:
# students = {'U19EE073': ['PRABHAKAR', '5'],
#  'U19ME114': ['MANISH', '5'],
#  'U19ME224': ['DHRUTIK', '5'],
#  'U19EC029': ['GAURAV', '4'],
#  'U19ME222': ['CHIRAG', '4'],
#  'U19EE039': ['MUDIT', '4'],
#  'U19EC160': ['OZA', '4'],
#  'U19EC045': ['HEMIN', '4'],
#  'U19EC030': ['ANSHOO', '4'],
#  'U19ME082': ['TAKSH', '3'],
#  'U19EC063': ['POOJA', '2'],
#  'U19EE037': ['JANVI', '2'],
#  'U19CS040': ['BANSEEDHAR', '2'],
#  'U19CS044': ['JAY', '2'],
#  'U19CH016': ['PRACHI', '2'],
#  'U19ME133': ['JAIN', '2'],
#  'U19CS105': ['SHASHWATHA', '1'],
#  'U19EE060': ['PEEPAM', '1'],
#  'U19ME108': ['ATUL', '1'],
#  'U19EC009': ['PRAKHAR', '1'],
#  'U19EC157': ['RISHIK', '1'],
#  'U19ME125': ['DHEERENDRA', '1'],
#  'U19EE031': ['HARSH', '1'],
#  'U19EE033': ['HARSH', '1'],
#  'U19ME002': ['KARANKUMAR', '0']}

In [362]:
# update_local()

In [354]:
def top():
    stop_time = time.time()+5
    webdriver.ActionChains(driver).click().perform()

    while time.time() < stop_time:
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()  
        webdriver.ActionChains(driver).send_keys(Keys.HOME).perform()

In [299]:
# update_in_code()

In [694]:
from selenium.webdriver.support.ui import WebDriverWait

In [693]:
def get_bot(bot,refresh = True):
    flag = True
    if bot == 1: #main bot
        print('getting main bot')
        BOT = 'https://teams.microsoft.com/_#/school/conversations/AUTO?threadId=19:72818b8c80fb4045bf22a2f19e8af04b@thread.tacv2&ctx=channel'
    elif bot == -1: #private bot
        print('getting private bot')
        BOT = 'https://teams.microsoft.com/_#/school/conversations/BOT?threadId=19:84124691e65d4ebabaee9d78e41ee04f@thread.tacv2&ctx=channel'
    else:
        flag = False
        print("INVALID BOT CHOICE!!")
    if flag:
        time.sleep(2)
        driver.get(BOT)
        time.sleep(2)
        if refresh:
            driver.refresh()
    else:
        sys.exit()

In [344]:
update_in_code()

{'U19EE073': ['PRABHAKAR', 5],
 'U19ME114': ['MANISH', 5],
 'U19ME224': ['DHRUTIK', 5],
 'U19EC029': ['GAURAV', 4],
 'U19ME222': ['CHIRAG', 4],
 'U19EE039': ['MUDIT', 4],
 'U19EC160': ['OZA', 4],
 'U19EC045': ['HEMIN', 4],
 'U19EC030': ['ANSHOO', 4],
 'U19ME082': ['TAKSH', 3],
 'U19EC063': ['POOJA', 2],
 'U19EE037': ['JANVI', 2],
 'U19CS040': ['BANSEEDHAR', 2],
 'U19CS044': ['JAY', 2],
 'U19CH016': ['PRACHI', 2],
 'U19ME133': ['JAIN', 2],
 'U19CS105': ['SHASHWATHA', 1],
 'U19EE060': ['PEEPAM', 1],
 'U19ME108': ['ATUL', 1],
 'U19EC009': ['PRAKHAR', 1],
 'U19EC157': ['RISHIK', 1],
 'U19ME125': ['DHEERENDRA', 1],
 'U19EE031': ['HARSH', 1],
 'U19EE033': ['HARSH', 1],
 'U19ME002': ['KARANKUMAR', 0]}

In [345]:
students

{'U19EE073': ['PRABHAKAR', 5],
 'U19ME114': ['MANISH', 5],
 'U19ME224': ['DHRUTIK', 5],
 'U19EC029': ['GAURAV', 4],
 'U19ME222': ['CHIRAG', 4],
 'U19EE039': ['MUDIT', 4],
 'U19EC160': ['OZA', 4],
 'U19EC045': ['HEMIN', 4],
 'U19EC030': ['ANSHOO', 4],
 'U19ME082': ['TAKSH', 3],
 'U19EC063': ['POOJA', 2],
 'U19EE037': ['JANVI', 2],
 'U19CS040': ['BANSEEDHAR', 2],
 'U19CS044': ['JAY', 2],
 'U19CH016': ['PRACHI', 2],
 'U19ME133': ['JAIN', 2],
 'U19CS105': ['SHASHWATHA', 1],
 'U19EE060': ['PEEPAM', 1],
 'U19ME108': ['ATUL', 1],
 'U19EC009': ['PRAKHAR', 1],
 'U19EC157': ['RISHIK', 1],
 'U19ME125': ['DHEERENDRA', 1],
 'U19EE031': ['HARSH', 1],
 'U19EE033': ['HARSH', 1],
 'U19ME002': ['KARANKUMAR', 0]}

In [374]:
_,ids = get_message_stuff()

In [375]:
ids

['m1591604091183',
 'm1591618310701',
 'm1591835029919',
 'm1592011368884',
 'm1592011454067',
 'm1592011706155',
 'm1592011774056',
 'm1592012184246',
 'm1592012512240',
 'm1592013614116',
 'm1592007586202',
 'm1592013631964']

In [372]:
ids

['m1591604091183',
 'm1591618310701',
 'm1591833249480',
 'm1591833256475',
 'm1591833259877',
 'm1591833269443',
 'm1591833381070',
 'm1591833384984',
 'm1591833395136',
 'm1591833398410',
 'm1591833406069',
 'm1591833416414',
 'm1591833421991',
 'm1591833425889',
 'm1591833519872',
 'm1591833856718',
 'm1591833860313',
 'm1591833867772',
 'm1591833871744',
 'm1591834962344',
 'm1591834966103',
 'm1591835025498',
 'm1591835029919',
 'm1592010441078',
 'm1592011368884',
 'm1592011454067',
 'm1592011706155',
 'm1592011774056',
 'm1592012184246',
 'm1592012512240',
 'm1592013614116',
 'm1592007586202',
 'm1592013631964',
 'm1592013790649',
 'm1592013928599',
 'm1592013944858']

In [370]:
ids[1]

'm1591618310701'

In [ ]:
while True:
    driver.find_element_by_css_selector(f'{ids[1]} > ')

In [288]:
time.time()

1592011225.727703

In [291]:
#m1591835029919 > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > see-more:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)

In [ ]:
#m1591833249480 > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > see-more:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)

In [33]:
# students = {}\

In [34]:
# students['U19EE073']= ['PRABHAKAR',5]
# students['U19ME114']= ['MANISH',5]
# students['U19ME224']= ['DHRUTIK',5]
# students['U19EC029']= ['GAURAV',4]
# students['U19ME222']= ['CHIRAG',4]
# students['U19EE039']= ['MUDIT',4]
# students['U19EC160']= ['OZA',4]
# students['U19EC045']= ['HEMIN',4]
# students['U19EC030']= ['ANSHOO',4]
# students['U19ME082']= ['TAKSH',3]
# students['U19EC063']= ['POOJA',2]
# students['U19EE037']= ['JANVI',2]
# students['U19CS040']= ['BANSEEDHAR',2]
# students['U19CS044']= ['JAY',2]
# students['U19CH016']= ['PRACHI',2]
# students['U19ME133']= ['JAIN',2]
# students['U19CS105']= ['SHASHWATHA',1]
# students['U19EE060']= ['PEEPAM',1]
# students['U19ME108']= ['ATUL',1]
# students['U19EC009']= ['PRAKHAR',1]
# students['U19EC157']= ['RISHIK',1]
# students['U19ME125']= ['DHEERENDRA',1]
# students['U19EE031']= ['HARSH',1]
# students['U19EE033']= ['HARSH',1]
# students['U19ME002']= ['KARANKUMAR',0]

In [35]:
# with open('students.pickle','wb') as f:
#     pickle.dump(students,f)

In [468]:
# driver.find_elements_by_class_name('thread-action-reply-message')[-1].click()

In [417]:
# AUTO STATS

# AUTO STOP

# AUTO SET_CHECK_PERIOD 3

# AUTO REMOVE

# AUTO REFRESH

# AUTO SWITCH_BOT


In [537]:
# def edit(message_bodies,message_ids,bot):
# #     try:
#     driver.execute_script("window.scrollTo(0, 0);")
#     print('trying to edit')
#     message_to_edit = message_bodies[INDEX_OF_MESSAGE]
#     print(f'message to edit: {message_to_edit}\n Scrolling message into view....')
#     driver.execute_script("return arguments[0].scrollIntoView(true);", message_to_edit)
#     print('scrolled successfully, trying to click on body...')
#     driver.find_element_by_xpath("//body").click() #click somewhere else on the body
#     time.sleep(0.5)
#     print('hovering over message..')
#     hover = ActionChains(driver).move_to_element(message_to_edit)
#     hover.perform()
#     time.sleep(0.5)
#     print('hovered over message..')
#     driver.find_element_by_css_selector('.message-actions-more').click()#click on "..."
#     time.sleep(0.5)
#     driver.find_element_by_css_selector("li.message-action-overflow-menu-item:nth-child(2) > button:nth-child(1) > span:nth-child(2)").click()#select edit      
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# #     except:
# #         get_bot(bot = bot,refresh = True)
# #         message_bodies,message_ids = get_message_stuff()
# #         message_bodies,message_ids = edit(message_bodies,message_ids,bot)
#     return message_bodies,message_ids
        

In [538]:
# students['U19EE033'] = ['HARSH','1']
# students['U19ME224'] = ['DHRUTIK','5']
# students['U19ME133'] = ['JAIN','2']
# students['U19ME082'] = ['TAKSH','3']
# students['U19CS040'] = ['BANSEEDHAR','2']

In [539]:
# students

In [541]:
# update_in_code()
# update_local()

In [540]:
# from selenium.webdriver.common.keys import Keys
# html = driver.find_element_by_tag_name('html')
# html.send_keys(Keys.HOME)

In [542]:
# driver.execute_script("window.scrollTo(0, 0)")

In [543]:
# departments = [re.split('U19([A-Z][A-Z])',key)[1] for key in students]
# departments

In [544]:
# list(Counter(departments).values())

In [545]:
# percentages = list(map(lambda x: (x/len(students))*100,list(Counter(departments).values())))
# percentages

In [546]:
# percentages = [str(x) for x in list(np.round(percentages,3))]
# percentages

In [547]:
# departments_list

In [548]:
# percentages = list(map(lambda x: (x/len(students))*100,list(Counter(departments).values())))
# percentages = [str(x) for x in list(np.round(percentages,3))]
# departments_list = list(Counter(departments).keys())
# stats = '\n'.join([departments_list[i]+' --> '+str(list(Counter(departments).values())[i])+' --> '+percentages[i]+'%' for i in range(0,len(departments_list)) ])
# stats = f"Active Students: {len(students)}\nDepartment wise:\n"+stats

In [549]:
# print(stats)

In [550]:
# keys 

In [551]:
# students

In [552]:
# def add_prefix(string):
#     string = "AUTO =>\n"+string
#     return string

In [553]:
# print(payload)

In [191]:
# driver.find_elements_by_class_name('thread-action-reply-message')[-1].click()
# in_htlm = str(driver.find_element_by_class_name('ts-edit-box').get_attribute("innerHTML"))
# cke_element = re.findall(r'<div id="(cke_[0-9]*_contents)"',in_htlm)[0]
# cke_element = driver.find_element_by_id(cke_element)

In [554]:
# cke_element.send_keys(Keys.CONTROL, "a")

# cke_element.send_keys(Keys.DELETE)

# cke_element.send_keys(Keys.CONTROL, "v")

# cke_element.send_keys(Keys.RETURN)


In [555]:
# cke_element = driver.find_element_by_css_selector(cke_wysiwyg_div)

In [556]:
# '>\n\n\nADMIN_COMMAND STOP\n\n, reply from'

In [694]:
# inner_html

In [557]:
# (1/(0.69*100*0.1))*1000

In [558]:
# ">\n\n\nADMIN_COMMAND STOP\n\n, reply from"

In [699]:
# inner_html

In [559]:
# driver.execute_script("window.scrollTo(0, 0);")

In [561]:
# SCROLL_PAUSE_TIME = 0.5

# # Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, 0);")

#     # Wait to load page
#     time.sleep(SCROLL_PAUSE_TIME)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

In [562]:
# send_reply(payload,bot)

In [563]:
# message_bodies,message_ids = get_message_stuff()
# inner_html = driver.find_element_by_id(message_ids[-1]).get_attribute('innerHTML')


In [564]:
# command

In [565]:
# soup.find('class','screen-reader-text')

In [700]:
# message_bodies,message_ids = get_message_stuff()

# inner_html = driver.find_element_by_id(message_ids[-1]).get_attribute('innerHTML')
# re.findall(">[^A-Z 0-9_]*([A-Z 0-9_]*).*, reply from",inner_html,re.DOTALL)

In [566]:
# re.findall("^>\s+([A-Z]\w+ \w+)\s+, reply from",inner_html,re.DOTALL)

In [567]:
# re.match(">, reply from",inner_html,re.DOTALL)